In [1]:
import torch
import sys
import os
from pathlib import Path
import numpy as np
from typing import List
from PIL import Image
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

sys.path.append(".")
sys.path.append("..")
sys.path.append(os.path.abspath('D:/Docs/Kursach/stylegan3-editing/utils/'))

from models.stylegan3.model import SG3Generator
from common import make_transform, tensor2im
from editor.Editor import FaceEditor
from models.stylegan3.model import GeneratorType

In [2]:
generator = SG3Generator(checkpoint_path="../pretrained_models/stylegan3-r-ffhq-1024x1024.pkl").decoder

num_images_per_cluster = 5
trunc = 0.5

INTERFACEGAN_RANGES = {
    "asian_eyes": (-4, 5),
    "age": (-5, 5),
    "smile": (-2, 2),
    "Male": (-2, 4)
}

Loading StyleGAN3 generator from path: ../pretrained_models/stylegan3-r-ffhq-1024x1024.pkl


Done!


In [3]:
csv_name = 'fruit'
df = pd.read_csv(f"{Path('./datasets/') / csv_name}.csv")
df = df.select_dtypes(exclude=['object'])
zero_row = pd.Series([0.5] * len(df.columns), index=df.columns)
df = pd.concat([pd.DataFrame([zero_row]), df], ignore_index=True)

In [4]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df.iloc[:])
df_scaled

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.14285714, 0.52973721, 0.86813187, 0.68      , 0.11627907],
       [0.14285714, 0.49654219, 0.82417582, 0.63      , 0.20930233],
       [0.14285714, 0.48547718, 0.75824176, 0.67      , 0.23255814],
       [0.42857143, 0.23651452, 0.62637363, 0.42      , 0.69767442],
       [0.42857143, 0.23098202, 0.6043956 , 0.41      , 0.6744186 ],
       [0.42857143, 0.21991701, 0.58241758, 0.38      , 0.62790698],
       [0.42857143, 0.21991701, 0.59340659, 0.38      , 0.72093023],
       [0.42857143, 0.20885201, 0.58241758, 0.35      , 0.72093023],
       [0.14285714, 0.49100968, 0.72527473, 0.73      , 0.97674419],
       [0.14285714, 0.47441217, 0.75824176, 0.65      , 0.90697674],
       [0.14285714, 0.45781466, 0.7032967 , 0.68      , 1.        ],
       [0.14285714, 0.47441217, 0.72527473, 0.71      , 0.97674419],
       [0.14285714, 0.42461964, 0.71428571, 0.66      , 0.88372093],
       [0.14285714, 0.45228216, 0.

In [5]:
num_samples, num_atributes = df_scaled.shape
df_latents = np.pad(df_scaled, ((0, 0), (0, 512-num_atributes)), mode='constant')
df_latents

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.14285714, 0.52973721, 0.86813187, ..., 0.        , 0.        ,
        0.        ],
       [0.14285714, 0.49654219, 0.82417582, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.31950207, 0.59340659, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.41908714, 0.65934066, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.32503458, 0.61538462, ..., 0.        , 0.        ,
        0.        ]])

In [26]:
cluster_images = []
for latent in df_latents:
    z = torch.from_numpy(latent).float().to('cuda').unsqueeze(0)
    w = generator.mapping(z, None, truncation_psi=trunc)
    with torch.no_grad():
        img = generator.synthesis(w, noise_mode='const')
        cluster_images.append(tensor2im(img[0]))

start_path = Path(f'./edit_results/{csv_name}')
start_path.mkdir(exist_ok=True, parents=True)
for i, img in enumerate(cluster_images):
    img.save(start_path / f"data_in_image_{i+1}.png")